# Ebay Car Sales Project- Nadir Nibras


## The aim of this project is to clean the data and analyze the included used car listings

In [3]:
import pandas as pd
import numpy as np

autos = pd.read_csv("autos.csv", encoding= "Latin-1");

In [4]:
autos;

In [5]:
autos.info();
autos.head();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 20 columns):
dateCrawled            50000 non-null object
name                   50000 non-null object
seller                 50000 non-null object
offerType              50000 non-null object
price                  50000 non-null object
abtest                 50000 non-null object
vehicleType            44905 non-null object
yearOfRegistration     50000 non-null int64
gearbox                47320 non-null object
powerPS                50000 non-null int64
model                  47242 non-null object
odometer               50000 non-null object
monthOfRegistration    50000 non-null int64
fuelType               45518 non-null object
brand                  50000 non-null object
notRepairedDamage      40171 non-null object
dateCreated            50000 non-null object
nrOfPictures           50000 non-null int64
postalCode             50000 non-null int64
lastSeen               50000 non-null obj

It seems that certain cells such as vehicle type and model have a lot of null values while most variables have no null values

In [6]:
autos.columns;

In [7]:
autos.columns=['date_crawled', 'name', 'seller', 'offerType', 'price', 
              'abtest', 'vehicleType', 'registration_year', 
              'gearbox', 'powerPS', 'model', 'odometer', 
              'registration_month', 'fuelType', 'brand',
              'unrepaired_damage', 'ad_created', 'nrOfPictures', 
              'postalCode', 'last_seen'];
autos.head;

The column names were changed so that all column names are now in a similar format and we can clean it easily

In [8]:
autos.describe(include='all');

seller and offer type have mostly 1 value

price, odometer need to be converted to numeric columns

In [9]:
autos["price"]=autos["price"].str.replace(",","").str.replace("$","").astype(float)
autos["odometer"]=autos["odometer"].str.replace(",","").str.replace("km","").astype(float)
autos= autos.rename(columns={"odometer": "odometer_km"});
autos;

In [10]:
autos["odometer_km"].unique().shape;
autos["odometer_km"].value_counts();
autos["odometer_km"].describe();


In [11]:
autos["price"].unique().shape;
#Removing unrealistical high/low prices
autos= autos[autos["price"].between(100,1000000)];
autos["price"].describe();
autos["price"].value_counts().sort_index(ascending=False);

A lot of cars are being sold for really low prices

In [12]:
autos['date_crawled'].str[:10].value_counts(normalize=True, dropna=False);

A lot of the dates seem to be in March or April of 2016. Not a lot of missing values

In [13]:
autos['ad_created'].str[:10].value_counts(normalize=True, dropna=False);

A lot of the dates seem to be in March or April of 2016. Not a lot of missing values

In [14]:
autos['last_seen'].str[:10].value_counts(normalize=True, dropna=False).sort_index();

A lot of the dates seem to be in March or April of 2016. Not a lot of missing values

In [15]:
autos["registration_year"].describe();

There are impossible values of registration year both on the minimum and the maximum end of things. Not all data points have a registration value

The highest and lowest acceptable values for registration year should be 2016 and 1900 respectively

In [16]:
df= autos[(autos["registration_year"]<1900)|(autos["registration_year"]>2016)];
df.shape

(1872, 20)

In [17]:
autos= autos[(autos["registration_year"].between(1900,2017))]
autos["registration_year"].value_counts();

The most common registration years seem to be the early 2000s but there are also a lot of cars in the late 90s

In [18]:
#Ranking cars by how often they are found in the overall data-set
autos["brand"].value_counts(normalize=True).index;

In [19]:
mcb=['volkswagen', 'bmw', 'opel', 'mercedes_benz', 'audi',
'ford', 'renault', 'peugeot', 'fiat', 'seat']


The most common 10 brands of cars up for sale were saved in a series

In [20]:
brand_mean_price={}
for brand in mcb:
    df= autos[autos["brand"]==brand]
    mean_price= df["price"].mean()
    brand_mean_price[brand]= mean_price

In [21]:
brand_mean_price;

Unsurprisingly, Ford and Open have cheaper average for their cars. The brands that are known for having expensive cars such as Audi and BMW do seem to have the highest mean prices.

In [22]:
brand_mean_milage= {}
for brand in mcb:
    df= autos[autos["brand"]==brand]
    mean_milage= df["odometer_km"].mean()
    brand_mean_milage[brand]= mean_milage
brand_mean_milage

{'audi': 129519.18406993686,
 'bmw': 132787.92539896173,
 'fiat': 117203.73376623377,
 'ford': 124266.20648259304,
 'mercedes_benz': 131196.03139991278,
 'opel': 129476.33824670287,
 'peugeot': 127275.30077848549,
 'renault': 128297.40431148262,
 'seat': 122015.76576576577,
 'volkswagen': 129023.03376316567}

In [53]:
bmp_series= pd.Series(brand_mean_price)
bmm_series= pd.Series(brand_mean_milage)
df = pd.DataFrame(bmp_series, columns=['mean_price'])
df["mean_milage"]= bmm_series
df

,mean_price,mean_milage
audi,9286.365226,129519.184070
bmw,8331.746587,132787.925399
fiat,2813.182630,117203.733766
ford,4060.090036,124266.206483
mercedes_benz,8572.807239,131196.031400
opel,2983.042281,129476.338247
peugeot,3090.864827,127275.300778
renault,2462.856137,128297.404311
seat,4388.873874,122015.765766
volkswagen,5580.219510,129023.033763


From a quick overview, there doesn't seem to be a correlation between mean milage and car brand price